<a href="https://colab.research.google.com/github/vishnunagar109/Machine_Learning/blob/main/Traffic%20Lights%20Detection/RPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
import os
import sys
import cv2
import pandas as pd 
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import random
from shutil import copyfile

In [ ]:
root_dir = 'gdrive/MyDrive/Datasets/OID'

In [ ]:
# # #Getting subset of the data for experimenting

images = pd.read_csv(os.path.join(root_dir,'train_images.csv'))
annotations = pd.read_csv(os.path.join(root_dir,'train_annotations.csv'))
# f1 = image_file.iloc[[i for i in range(100)]]
# f2 = annotations_file.iloc[[i for i in range(100)]]
# f1.to_csv(os.path.join(root_dir,'train_images_sub.csv'))
# f2.to_csv(os.path.join(root_dir,'train_annotations_sub.csv'))

In [ ]:
# images = pd.read_csv(os.path.join(root_dir,'train_images.csv'))
# annotations = pd.read_csv(os.path.join(root_dir,'train_annotations.csv'))
classes = pd.read_csv(os.path.join(root_dir,'class_description.csv'),header=None)

In [ ]:
images.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
2,7e8584b0f487cb9e,train,https://c7.staticflickr.com/8/7056/7143870979_...,https://www.flickr.com/photos/circasassy/71438...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/circasassy/,CircaSassy,A Christmas carol and The cricket on the heart...,2371584,3hQwu0iSzY1VIoXiwp0/Mg==,https://c7.staticflickr.com/8/7056/7143870979_...,0.0
3,86638230febe21c4,train,https://farm5.staticflickr.com/5128/5301868579...,https://www.flickr.com/photos/ajcreencia/53018...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ajcreencia/,Alex,Abbey and Kenny,949267,onB+rCZnGQg5PRX7xOs18Q==,https://c4.staticflickr.com/6/5128/5301868579_...,NaN
4,249086e72671397d,train,https://c6.staticflickr.com/4/3930/15342460029...,https://www.flickr.com/photos/codnewsroom/1534...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/codnewsroom/,COD Newsroom,Suburban Law Enforcement Academy 20th Annivers...,6541758,MjpaAVbMAWbCusSaxI1D7w==,https://c1.staticflickr.com/4/3930/15342460029...,0.0


In [ ]:
def draw_bbox(img_id):
  img_url = images.loc[images["ImageID"]==img_id]['OriginalURL'].values[0]
  img = io.imread(img_url)
  print(img.shape)
  # io.imshow(img)
  #h,w,c = img.shape   #width,height,channels
  bboxes = annotations[annotations["ImageID"]==img_id]
  for idx,row in bboxes.iterrows():
    xmin = int(row['XMin']*img.shape[0])
    xmax = int(row['XMax']*img.shape[0])
    ymin = int(row['YMin']*img.shape[1])
    ymax = int(row['YMax']*img.shape[1])
    label = row['LabelName']
    classs = classes[classes[0]==label]
    class_name = classs[1].values[0]
    print(f"Coordinates: ({xmin},{ymin}),({xmax},{ymax})")
    cv2.rectangle(img,(xmin,ymin),(xmax,ymax),(255,0,0),5)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img,class_name,(xmin,ymin-10),font,1,(0,255,0),5)
  plt.figure(figsize=(15,10))
  plt.title("bbox")
  plt.imshow(img)
  plt.axis('off')
  plt.show()


In [ ]:
# draw_bbox("b37f763ae67d0888")

In [ ]:
print(classes[1].count())

601


In [ ]:
# beds = classes[classes[1]=="Bed"]
# chairs = classes[classes[1]=="Chair"]
# tables = classes[classes[1]=="Table"]
lights=classes[classes[1]=="Traffic light"]

In [ ]:
# label_bed = beds[0].values[0]
# chair_label = chairs[0].values[0]
# table_label = tables[0].values[0]
lights_label=lights[0].values[0]

'/m/015qff'

In [ ]:
# bed_bbox = annotations[annotations["LabelName"]==label_bed]
# chair_bbox = annotations[annotations["LabelName"]==chair_label]
# table_bbox = annotations[annotations["LabelName"]==table_label]
# print(f"Total instances of beds: {len(bed_bbox)}")
# print(f"Total instances of chairs: {len(chair_bbox)}")
# print(f"Total instances of tables: {len(table_bbox)}")
lights_bbox = annotations[annotations["LabelName"]==lights_label]
print(len(lights_bbox))

7426


In [ ]:
# chair_bbox

In [ ]:
# bed_img_ids = bed_bbox['ImageID']
# chair_img_ids = chair_bbox['ImageID']
# table_img_ids = table_bbox['ImageID']
lights_img_ids = lights_bbox['ImageID']

In [ ]:
# n=10 #selecting 10 random images for training
# subbed_img_ids = random.sample(list(bed_img_ids),n)
# subchair_img_ids = random.sample(list(chair_img_ids),n)
# subtable_img_ids = random.sample(list(table_img_ids),n)

In [ ]:
# #select 10 images from images dataframe
# subbed_images = images.loc[images['ImageID'].isin(subbed_img_ids)]
# subchair_images = images.loc[images['ImageID'].isin(subchair_img_ids)]
# subtable_images = images.loc[images['ImageID'].isin(subtable_img_ids)]

In [ ]:
# #converting data into dictionary of form -> {'ImageID':'OriginalURL'}
# subbed_dict = subbed_images[['ImageID','OriginalURL']].set_index('ImageID')["OriginalURL"].to_dict()
# subchair_dict = subchair_images[['ImageID','OriginalURL']].set_index('ImageID')["OriginalURL"].to_dict()
# subtable_dict = subtable_images[['ImageID','OriginalURL']].set_index('ImageID')["OriginalURL"].to_dict()
# mappings = [subbed_dict,subchair_dict,subtable_dict]
lights_images = images.loc[images['ImageID'].isin(lights_img_ids)]
lights_dict=lights_images[['ImageID','OriginalURL']].set_index('ImageID')["OriginalURL"].to_dict()

In [ ]:
# #Download images
# c = ['bed','chair','table']  #classes we are working on
# for idx,obj in enumerate(c):
#   n_issues=0
#   root_path = os.path.join(root_dir,obj)
#   if not os.path.exists(root_path):
#     os.mkdir(root_path)
#     for img_id,url in mappings[idx].items():
#       try:
#         img = io.imread(url)
#         saved_path = os.path.join(root_path,img_id+".jpg")
#         io.imsave(saved_path,img)
#       except Exception as e:
#         n_issues+=1
    
#   print(f"Images issues: {n_issues}")
n_issues=0
root_path = os.path.join(root_dir,'lights')
if not os.path.exists(root_path):
     os.mkdir(root_path)
     for img_id,url in lights_dict.items():
       try:
         img = io.imread(url)
         saved_path = os.path.join(root_path,img_id+".jpg")
         io.imsave(saved_path,img)
       except Exception as e:
         n_issues+=1
    
print(f"Images issues: {n_issues}")

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1310720 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262144 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131072 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)


Images issues: 125


In [ ]:
# #Save images into train and test directories
train_path = os.path.join(root_dir,'train_lights')
test_path = os.path.join(root_dir,'test_lights')
os.mkdir(train_path)
os.mkdir(test_path)

In [ ]:
# #copy data into train and test directories
# for i in range(len(c)):
#   all_img = os.listdir(os.path.join(root_dir,c[i]))
#   all_img = [f for f in all_img if not f.startswith('.')]
#   random.shuffle(all_img)
#   limit = int(n*0.8)  #80% images for training
#   train_img = all_img[:limit]
#   test_img = all_img[limit:]

#   for j in range(len(train_img)):
#     original_path = os.path.join(os.path.join(root_dir,c[i]),train_img[j])
#     new_path = os.path.join(train_path,train_img[j])
#     copyfile(original_path,new_path)

#   for j in range(len(test_img)):
#     original_path = os.path.join(os.path.join(root_dir,c[i]),test_img[j])
#     new_path = os.path.join(test_path,test_img[j])
#     copyfile(original_path,new_path)

all_img = os.listdir(os.path.join(root_dir,'lights'))
n=len(all_img)
all_img = [f for f in all_img if not f.startswith('.')]
random.shuffle(all_img)
limit = int(n*0.88)  #80% images for training
train_img = all_img[:limit]
test_img = all_img[limit:]

for j in range(len(train_img)):
  original_path = os.path.join(os.path.join(root_dir,'lights'),train_img[j])
  new_path = os.path.join(train_path,train_img[j])
  copyfile(original_path,new_path)

for j in range(len(test_img)):
  original_path = os.path.join(os.path.join(root_dir,'lights'),test_img[j])
  new_path = os.path.join(test_path,test_img[j])
  copyfile(original_path,new_path)

In [ ]:
# labels = [label_bed,chair_label,table_label]
train_df = pd.DataFrame(columns=['FileName', 'XMin', 'YMin', 'XMax', 'YMax', 'ClassName'])

# Find boxes in each image and put them in a dataframe
train_imgs = os.listdir(train_path)
train_imgs = [name for name in train_imgs if not name.startswith('.')]

for i in range(len(train_imgs)):
    sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of boxes: ' + str(len(train_df)) + '\r')
    sys.stdout.flush()
    img_name = train_imgs[i]
    img_id = img_name[0:16]
    tmp_df = annotations[annotations['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['LabelName']
        if labelName==lights_label:
            train_df = train_df.append({'FileName': img_name, 
                                        'XMin': row['XMin'], 
                                        'YMin': row['YMin'], 
                                        'XMax': row['XMax'], 
                                        'YMax': row['YMax'], 
                                        'ClassName': 'lights'}, 
                                        ignore_index=True)

In [ ]:
train_df.head()

,FileName,XMin,YMin,XMax,YMax,ClassName
0,0470f9aa3330efcb.jpg,0.004688,0.797917,0.023438,0.845833,lights
1,0470f9aa3330efcb.jpg,0.129688,0.879167,0.178125,0.897917,lights
2,0470f9aa3330efcb.jpg,0.260938,0.910417,0.273438,0.939583,lights
3,0470f9aa3330efcb.jpg,0.512500,0.591667,0.542188,0.668750,lights
4,0470f9aa3330efcb.jpg,0.606250,0.016667,0.743750,0.375000,lights


In [ ]:
# labels = [label_bed,chair_label,table_label]
test_df = pd.DataFrame(columns=['FileName', 'XMin', 'YMin', 'XMax', 'YMax', 'ClassName'])

# Find boxes in each image and put them in a dataframe
test_imgs = os.listdir(test_path)
test_imgs = [name for name in test_imgs if not name.startswith('.')]

for i in range(len(test_imgs)):
    sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of boxes: ' + str(len(test_df)) + '\r')
    sys.stdout.flush()
    img_name = test_imgs[i]
    img_id = img_name[0:16]
    tmp_df = annotations[annotations['ImageID']==img_id]
    for index, row in tmp_df.iterrows():
        labelName = row['LabelName']
        if labelName==lights_label:
            test_df = test_df.append({'FileName': img_name, 
                                        'XMin': row['XMin'], 
                                        'YMin': row['YMin'], 
                                        'XMax': row['XMax'], 
                                        'YMax': row['YMax'], 
                                        'ClassName': 'lights'}, 
                                        ignore_index=True)

In [ ]:
train_df.to_csv(os.path.join(root_dir,'train_lights.csv'))
test_df.to_csv(os.path.join(root_dir,'test_lights.csv'))

In [ ]:
train_df = pd.read_csv(os.path.join(root_dir,'train_lights.csv'))
with open(os.path.join(root_dir,"train_annotations.txt"),"w+") as f:
  for idx,row in train_df.iterrows():
    img = cv2.imread(root_dir+"/train_lights/"+row['FileName'])
    h,w = img.shape[:2]
    x1 = int(row['XMin']*w)
    x2 = int(row['XMax']*w)
    y1 = int(row['YMin']*h)
    y2 = int(row['YMax']*h)
    train_file_path = root_dir+"/train_lights"
    filename = os.path.join(train_file_path,row['FileName'])
    c = row['ClassName']
    f.write(filename+','+str(x1)+','+str(y1)+','+str(x2)+','+str(y2)+','+c+'\n')

In [ ]:
test_df = pd.read_csv(os.path.join(root_dir,'test_lights.csv'))
with open(os.path.join(root_dir,"test_annotations.txt"),"w+") as f:
  for idx,row in test_df.iterrows():
    img = cv2.imread(root_dir+"/test_lights/"+row['FileName'])
    h,w = img.shape[:2]
    x1 = int(row['XMin']*w)
    x2 = int(row['XMax']*w)
    y1 = int(row['YMin']*h)
    y2 = int(row['YMax']*h)
    train_file_path = root_dir+"/test_lights"
    filename = os.path.join(train_file_path,row['FileName'])
    c = row['ClassName']
    f.write(filename+','+str(x1)+','+str(y1)+','+str(x2)+','+str(y2)+','+c+'\n')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [ ]:
#backbone vgg network
vgg = models.vgg16(pretrained=True)
f = list(vgg.features)
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
req_features=[]
dummy_img = torch.zeros((1, 3, 800, 800)).float()
k=dummy_img.clone()
for i in f:
  k=i(k)
  if(k.size()[2]<800//16):
    break
  req_features.append(i)
  out_c=k.size()[1]
print(len(req_features))
print(out_c)

30
512


In [ ]:
print(k.size()[2])

25


In [ ]:
#backbone 
backbone = nn.Sequential(*req_features)
print(backbone)
out = backbone(dummy_img)
print(out.shape)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [20]:
#anchor boxes
ratios=[0.5,1,2]
anchor_scales=[8,16,32]
sub_sample=16
feature_size=800//sub_sample
anchor_base=np.zeros((len(ratios)*len(anchor_scales),4),dtype=np.float32)

In [21]:
ctr_x = np.arange(16,(feature_size+1)*16,16)
ctr_y = np.arange(16,(feature_size+1)*16,16)
idx=0
centers=[]
for i in range(len(ctr_x)):
  for j in range(len(ctr_y)):
    y=ctr_y[j]-8
    x=ctr_x[i]-8
    centers.append([x,y])
    idx+=1
centers=np.array(centers)
centers.shape

(2500, 2)

In [26]:
anchors=np.zeros((feature_size*feature_size*9,4))
idx=0
for c in centers:
  x,y=c
  for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
      h=sub_sample*anchor_scales[j]*np.sqrt(ratios[i])
      w=sub_sample*anchor_scales[j]*np.sqrt(1/ratios[i])
      anchors[idx,0]=x-w/2
      anchors[idx,1]=y-h/2
      anchors[idx,2]=x+w/2
      anchors[idx,3]=y+h/2
      idx+=1
anchors.shape

(22500, 4)

In [27]:
#exclude anchor boxes that are outside the image
idx_inside=np.where((anchors[:,0]>=0)&(anchors[:,1]>=0)&(anchors[:,2]<=800)&(anchors[:,3]<=800))
valid_boxes=anchors[idx_inside]
valid_boxes.shape

(8940, 4)

In [ ]:
class RPN(nn.Module):
  def __init__(self,num_anchors):
    super(RPN,self).__init__()
    self.conv = nn.Conv2d(512,512,(3,3),stride=(1,1),padding=(1,1))
    self.classifier = nn.Conv2d(512,num_anchors,(1,1))
    self.regressor = nn.Conv2d(512,num_anchors*4,(1,1))
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self,base_out):
    x = self.conv(base_out)
    x = self.relu(x)
    x_class = self.classifier(x)
    x_class = self.sigmoid(x_class)
    x_regress = self.regressor(x)

    return [x_class,x_regress]

In [ ]:
#test RPN
test_RPN = RPN(9)
[out1,out2] = test_RPN(out)
print(out1.shape)
print(out2.shape)

torch.Size([1, 9, 7, 7])
torch.Size([1, 36, 7, 7])


In [ ]:

#Overall Model
class Net(nn.Module):
  def __init__(self,in_channels,num_anchors=9):
    super(Net,self).__init__()
    self.in_channels = in_channels
    self.num_anchors = num_anchors
    self.vgg = VGG16(self.in_channels)
    self.rpn = RPN(self.num_anchors)

  def forward(self,x):
    base_out = self.vgg(x)
    [x_c,x_r] = self.rpn(base_out)

    return [x_c,x_r,base_out]

In [ ]:
#test overall Network
model = Net(3,9)
x = torch.randn(1,3,224,224)
[o1,o2,o3] = model(x)
print(o1.shape)
print(o2.shape)
print(o3.shape)

torch.Size([1, 9, 7, 7])
torch.Size([1, 36, 7, 7])
torch.Size([1, 512, 7, 7])


In [ ]:
#ROI Pooling
class ROI_Pool(nn.Module):
  def __init__(self,pool_size,num_rois,**kwargs):
    super(ROI_Pool,self).__init__()
    self.pool_size = pool_size
    self.num_rois = num_rois
    